In [3]:
import pandas as pd

In [5]:
airline_flight_routes = pd.read_csv("Resources/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets_20240712.csv")


/var/folders/mx/dfq09dqx26b4x9cn21h8h77r0000gn/T/ipykernel_52612/2138598550.py:1: DtypeWarning: Columns (20,21) have mixed types. Specify dtype option on import or set low_memory=False.
  airline_flight_routes = pd.read_csv("Resources/Consumer_Airfare_Report__Table_1a_-_All_U.S._Airport_Pair_Markets_20240712.csv")
